In [1]:
import xarray as xr
import numpy as np

In [2]:
def calc_core(hh,pre,lat,lon):
    cores = []
    pre_sub = np.zeros((5,5))
    N_row,N_col = np.shape(pre)
    for i in range(2,N_row-4,1):
        for j in range(2,N_col-4,1):
            pre_sub = pre[i-2:i+3,j-2:j+3]
            if pre_sub[2,2] == np.max(pre_sub) and pre_sub[2,2]>=5 :
                cores.append((hh,lat[i],lon[j]))
    cores = np.array(cores)  
    return cores



In [3]:
icon_path = '/scratch/nf33/hk25_DOCmeso/ICON_interp/'
UM_path = '/scratch/nf33/hk25_DOCmeso/UM_interp/'

In [4]:
f1 = xr.open_dataset(icon_path + 'ICON_pr_z10_2020-04-01_MarCont.nc')
pr_icon = f1['pr']
lat_icon = f1['lat']
lon_icon = f1['lon']
f1.close()

f2 = xr.open_dataset(UM_path + 'UM_pr_z10_2020-04-01_MarCont.nc')
pr_UM = f2['pr']
lat_UM = f2['lat']
lon_UM = f2['lon']
f2.close()

In [6]:
icon_cores_day = np.zeros((1,3))
for hh in range(2): # the first colomn of the txt is the order for snapshot.
    cores = calc_core(hh,pr_icon[hh,:,:]*3600,lat_icon,lon_icon)
    np.shape(cores)
    icon_cores_day = np.vstack((icon_cores_day, cores))


In [8]:
unique_icon_cores_day = np.unique(icon_cores_day[1:,:], axis=0)
np.savetxt('/g/data/w28/ym7079/Hackthon/DOC/hk25-AusNode-DOCmeso/get_metrics/Yinglin/2020-04-01.txt', unique_icon_cores_day[1:], fmt='%d %.6f %.6f', delimiter=' ')  

In [11]:
N = 673 # the first 673 rows is for the first snapshot
#ids = range(np.shape(unique_icon_cores_day)[0])  # should get the rows for the same snapshot. 
ids = range(N)
data = unique_icon_cores_day 
ds = xr.Dataset(
    {
        'time': (['id'], data[:N, 0]),
        'lat': (['id'], data[:N, 1]),
        'lon': (['id'], data[:N, 2]),
    },
    coords={'id': ids}
)

# 3. save as netcdf file
ds.to_netcdf('/g/data/w28/ym7079/Hackthon/DOC/hk25-AusNode-DOCmeso/get_metrics/Yinglin/2020-04-01.nc')